In [ ]:
# https://arxiv.org/abs/1603.05027 
# https://www.researchgate.net/publication/336642248_A_Deep_Learning_Approach_for_Automated_Diagnosis_and_Multi-Class_Classification_of_Alzheimer's_Disease_Stages_Using_Resting-State_fMRI_and_Residual_Neural_Networks#pf10

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.python.keras import backend as K
from keras.utils import plot_model

from sklearn.utils import shuffle

import os
from pathlib import Path
from IPython.display import Image
import multiprocessing

from utils.DomainImageGenerator import DomainImageGenerator
from utils.ResNet18 import build_resnet18

2025-07-09 12:04:39.571116: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-09 12:04:39.575234: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-09 12:04:39.587069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752055479.605789  501634 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752055479.611539  501634 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-09 12:04:39.631978: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
model = build_resnet18(input_shape=(32, 32, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Path("build").mkdir(parents=True, exist_ok=True)
Path("build/ResNet18").mkdir(parents=True, exist_ok=True)

plot_model(model, to_file='build/ResNet18/Model.png', show_shapes=True, show_layer_names=True)
Image('build/ResNet18/Model.png')

2025-07-09 12:04:43.295792: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
# Load the Dataset as Generators so not all images are loaded into ram at the same time. The generator class loads the needed images per batch
cpu_count = multiprocessing.cpu_count()
njobs = int(cpu_count * 0.9) # Just set to use some percent of cpus
print(njobs)
max_queue_size = 15
use_multiprocessing = True

train_gen = DomainImageGenerator("prepared/train.csv", batch_size=64, img_size=(32, 32), shuffle=True, n_jobs=njobs, use_multiprocessing=use_multiprocessing, max_queue_size=max_queue_size) # "batch_size=64, img_size=(32, 32), shuffle=True]" are the default values
val_gen = DomainImageGenerator("prepared/valid.csv", batch_size=64, n_jobs=njobs, use_multiprocessing=use_multiprocessing, max_queue_size=max_queue_size)

43


In [5]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    verbose=1
)

Epoch 1/20
  72/1407 ━━━━━━━━━━━━━━━━━━━━ 11:09 501ms/step - accuracy: 0.6994 - loss: 0.9110

Process Keras_worker_ForkPoolWorker-37:
Process Keras_worker_ForkPoolWorker-43:
Process Keras_worker_ForkPoolWorker-36:
Process Keras_worker_ForkPoolWorker-7:
Process Keras_worker_ForkPoolWorker-27:
Process Keras_worker_ForkPoolWorker-17:
Process Keras_worker_ForkPoolWorker-19:
Process Keras_worker_ForkPoolWorker-22:
Process Keras_worker_ForkPoolWorker-10:
Process Keras_worker_ForkPoolWorker-39:
Process Keras_worker_ForkPoolWorker-12:
Process Keras_worker_ForkPoolWorker-35:
Process Keras_worker_ForkPoolWorker-24:
Process Keras_worker_ForkPoolWorker-34:
Process Keras_worker_ForkPoolWorker-33:
Process Keras_worker_ForkPoolWorker-11:
Process Keras_worker_ForkPoolWorker-21:
Process Keras_worker_ForkPoolWorker-31:
Process Keras_worker_ForkPoolWorker-16:
Process Keras_worker_ForkPoolWorker-6:
Process Keras_worker_ForkPoolWorker-28:
Process Keras_worker_ForkPoolWorker-13:
Process Keras_worker_ForkPoolWorker-29:
Process Keras_worker_ForkPoolWorker-1:
Process Keras_worker_ForkPoolWorker-30:
Pro

KeyboardInterrupt: 

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper right')
plt.savefig("build/ResNet18/Loss.png")

In [ ]:
model.save("../models/ResNet18.keras")

import json
model.save_weights("../models/ResNet18.weights.json", max_shard_size=0.095) # smaller then 95MB
json_string = model.to_json()
with open('../models/ResNet18_config.json', 'w', encoding='utf-8') as f:
    json.dump(json_string, f, ensure_ascii=False, indent=4)